In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Underweight"
cohort = "GSE131835"

# Input paths
in_trait_dir = "../../input/GEO/Underweight"
in_cohort_dir = "../../input/GEO/Underweight/GSE131835"

# Output paths
out_data_file = "../../output/preprocess/Underweight/GSE131835.csv"
out_gene_data_file = "../../output/preprocess/Underweight/gene_data/GSE131835.csv"
out_clinical_data_file = "../../output/preprocess/Underweight/clinical_data/GSE131835.csv"
json_path = "../../output/preprocess/Underweight/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json

# 1. Gene Expression Data Availability
# From the background information, we can see this dataset uses Affymetrix Clariom S Microarray
# which is used for gene expression profiling, so gene expression data is available.
is_gene_available = True

# 2. Data Availability and Conversion Functions

# 2.1 Trait (Underweight)
# From the background information, we can see this study is about cancer cachexia,
# which is characterized by weight loss. The trait "Underweight" can be associated
# with the 'group' field at index 1, which has categories 'CWL' (Cancer Weight Loss),
# 'CWS' (Cancer Weight Stable), and 'CONTROL'.
trait_row = 1

def convert_trait(value):
    """Convert trait value to binary (0 or 1)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # CWL (Cancer Weight Loss) corresponds to underweight
    if value == 'CWL':
        return 1
    # CWS (Cancer Weight Stable) and CONTROL are not underweight
    elif value in ['CWS', 'CONTROL', 'CONTROl']:
        return 0
    else:
        return None

# 2.2 Age
# Age information is available in the sample characteristics at index 3
age_row = 3

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 2.3 Gender
# Gender information is available in the sample characteristics at index 2
gender_row = 2

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # The sample characteristics dictionary was shown in the previous output
    # This data is typically stored in the series_matrix.txt.gz file
    # We'll use the available matrix file path to load it
    
    # Load the data from the matrix file path
    matrix_file_path = os.path.join(in_cohort_dir, "GSE131835_series_matrix.txt.gz")
    
    # Access clinical data from the provided file
    # Since we're dealing with a GEO dataset, we'll use the appropriate tools
    # from the tools.preprocess module to load the clinical data
    clinical_data = pd.read_pickle(os.path.join("./output/parse", f"{cohort}_clinical_data.pkl"))
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    clinical_preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(clinical_preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# First, let's define data availability by analyzing the previous step output

# Since the previous step's output is empty, we need to load and analyze the data ourselves
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Let's first try to load the clinical data
clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
gene_data_file = os.path.join(in_cohort_dir, "gene_data.csv")

# Check if the files exist
is_gene_available = os.path.exists(gene_data_file)
is_clinical_data_available = os.path.exists(clinical_data_file)

# Load clinical data if available
if is_clinical_data_available:
    clinical_data = pd.read_csv(clinical_data_file)
    # Preview the clinical data to understand structure
    print("Clinical data preview:")
    print(clinical_data.head())
    
    # Check for unique values in each row to identify trait, age, and gender
    sample_characteristics = {}
    for i, row in clinical_data.iterrows():
        unique_values = row.drop(row.index[0]).unique()
        sample_characteristics[i] = list(unique_values)
        print(f"Row {i}: {list(unique_values)}")
else:
    print("Clinical data not available")
    clinical_data = None
    sample_characteristics = {}

# Based on the analysis, define trait_row, age_row, and gender_row
# Since we don't have output from the previous step, we'll make educated guesses
# These would be updated after seeing the actual data

# If we can't determine from the output, we'll set them to None initially
trait_row = None
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary based on underweight status
    value = value.lower()
    if 'underweight' in value or 'low bmi' in value:
        return 1
    elif 'normal' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to numeric age
    try:
        # Remove any non-numeric characters (except decimal point)
        numeric_value = ''.join(c for c in value if c.isdigit() or c == '.')
        return float(numeric_value)
    except:
        return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary gender
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Before finalizing, let's check if we have any clinical data and extract more information
if is_clinical_data_available and clinical_data is not None:
    # This is a placeholder - we would analyze actual data here
    # Without the actual data, we're making an educated guess
    
    # For simplicity, we'll assume trait_row is None until we can verify from the data
    # This indicates we don't have trait data currently
    pass

# Validate and save cohort info
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available and trait_row is not None, extract clinical features
if is_clinical_data_available and trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_data)
    print("Selected clinical data preview:")
    print(preview)
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_data.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Clinical feature extraction skipped - no trait data available")


### Step 4: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# These identifiers appear to be Ensembl gene IDs with "_at" suffix
# Ensembl IDs start with "ENSG" for human genes, but they're not standard human gene symbols
# They need to be mapped to standard gene symbols for better interpretability

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in gene annotation correspond to gene identifiers and gene symbols
# From previous outputs we see:
# - Gene expression data has identifiers like 'ENSG00000000003_at' in the index
# - Gene annotation has 'ID' column with same format ('ENSG00000000003_at')
# - The 'ORF' column in gene annotation contains gene symbols (e.g., 'TSPAN6')

# 2. Extract the gene mapping between identifiers and symbols
print("Creating gene mapping between identifiers and gene symbols...")
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='ORF')
print(f"Gene mapping shape: {gene_mapping_df.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Load gene expression data from previously saved file or re-extract it
import os
import pandas as pd

# Re-extract the gene data since variables from previous step are not available
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file_path)

# Get gene annotation and create mapping
gene_annotation = get_gene_annotation(soft_file_path)
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='ORF')
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Normalize gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print(f"First few gene symbols after normalization: {list(gene_data.index[:10])}")

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract and prepare clinical data
# SOFT file contains the sample characteristics
background_info, clinical_data = get_background_and_clinical_data(matrix_file_path)

# From the sample characteristics in Step 1, we know:
# - trait_row = 1 (group: CWL, CWS, CONTROL where CWL indicates weight loss)
# - age_row = 3 (age information)
# - gender_row = 2 (Sex information)

trait_row = 1
age_row = 3
gender_row = 2

def convert_trait(value):
    """Convert Cancer Weight Loss to underweight status (binary)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # CWL (Cancer Weight Loss) corresponds to underweight
    if value == 'CWL':
        return 1
    # CWS (Cancer Weight Stable) and CONTROL are not underweight
    elif value in ['CWS', 'CONTROL', 'CONTROl']:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical features extracted:")
print(preview_df(clinical_features))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(preview_df(linked_data))

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")

# 6. Conduct quality check and save the cohort information
note = "Dataset contains gene expression data from adipose tissue samples comparing cancer patients with weight loss to controls."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")